# **INFO5731 In-class Exercise 5**

**This exercise aims to provide a comprehensive learning experience in text analysis and machine learning techniques, focusing on both text classification and clustering tasks.**

***Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks***.

**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of the day tomorrow, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission, and no requests will be answered. Manage your time accordingly.**


## **Question 1 (20 Points)**

The purpose of the question is to practice different machine learning algorithms for **text classification** as well as the performance evaluation. In addition, you are requried to conduct **10 fold cross validation** (https://scikit-learn.org/stable/modules/cross_validation.html) in the training.



The dataset can be download from canvas. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data.


**Algorithms:**

*   MultinominalNB
*   SVM
*   KNN
*   Decision tree
*   Random Forest
*   XGBoost
*   Word2Vec
*   BERT

**Evaluation measurement:**


*   Accuracy
*   Recall
*   Precison
*   F-1 score


In [ ]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

'''
  MultinominalNB,
  SVM,
  KNN,
  DecisionTree,
  RandomForest,
  XGBoost,
  Word2Vec,
  BERT,
)
'''

from google.colab import drive
drive.mount('drive', force_remount=True)

train_review = []
test_review = []

with open("/content/drive/My Drive/exercise09_datacollection/stsa-train.txt") as tr_reviews:
  train_review = [[int(review[:2].rstrip()),review.rstrip()[2:len(review)]] for review in tr_reviews]

with open("/content/drive/My Drive/exercise09_datacollection/stsa-train.txt") as te_reviews:
  test_review = [[int(review[:2].rstrip()),review.rstrip()[2:len(review)]] for review in te_reviews]

df_train = pd.DataFrame(train_review, columns=["Sentiment","Review"])
df_test = pd.DataFrame(test_review, columns=["Sentiment","Review"])

cv = KFold(n_splits=10, random_state=1, shuffle=True)

#MultinominalNB
'''
count_vect = CountVectorizer()
X_train = count_vect.fit_transform(df_train["Review"])
y_text = count_vect.fit_transform(df_test["Review"])
#print(X_train)
clf = MultinomialNB()
scores = cross_val_score(clf, X_train, y_text, cv=10)

print(scores.mean())
'''

Mounted at drive


'\ncount_vect = CountVectorizer()\nX_train = count_vect.fit_transform(df_train["Review"])\ny_text = count_vect.fit_transform(df_test["Review"])\n#print(X_train)\nclf = MultinomialNB()\nscores = cross_val_score(clf, X_train, y_text, cv=10)\n\nprint(scores.mean())\n'

## **Question 2 (20 Points)**

The purpose of the question is to practice different machine learning algorithms for **text clustering**.

Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

**Apply the listed clustering methods to the dataset:**
*   K-means
*   DBSCAN
*   Hierarchical clustering
*   Word2Vec
*   BERT

You can refer to of the codes from  the follwing link below.
https://www.kaggle.com/karthik3890/text-clustering

In [ ]:
# Write your code here
import pandas as pd

from google.colab import drive
drive.mount('drive', force_remount=True)

#with open("/content/drive/My Drive/Amazon_Unlocked_Mobile") as rev:
phone_review = pd.read_csv("/content/drive/My Drive/Amazon_Unlocked_Mobile.csv")

print([phone_review.shape])

phone_review.head()


Mounted at drive
[(413840, 6)]


,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


**In one paragraph, please compare the results of K-means, DBSCAN, Hierarchical clustering, Word2Vec, and BERT.**

**Write your response here:**

.

.

.

.

.




# Mandatory Question

**Important: Reflective Feedback on this exercise**

Please provide your thoughts and feedback on the exercises you completed in this assignment.


**(Your submission will not be graded if this question is left unanswered)**



In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:





'''